In [1]:
#Initial environement of computations


import matplotlib.pyplot as plt
import numpy as np
from kanon.calendars import Calendar, Date
from kanon.calendars.calendars import float_to_hm, hm_to_float
from kanon.tables import HTable
from kanon.units import Sexagesimal, Historical
from kanon.units.precision import set_precision, TruncatureMode, PrecisionMode
import astropy.units as u


from kanon.tables.htable_reader import (
    DISHAS_REQUEST_URL,
    read_historical,
    read_sexag_array,
    read_intsexag_array,
)
from kanon.tables import HTable
import requests

from kanon.units import Sexagesimal, Historical, IntegerAndSexagesimal, Temporal
from kanon.calendars import*

Cal=julian_ad = Calendar.registry["Julian A.D."]

number_reader = {
    "sexagesimal": read_sexag_array,
    "floating sexagesimal": read_sexag_array,
    "integer and sexagesimal": read_intsexag_array,
    "historical": read_historical,
}

In [2]:
#Initial table set

# Historical target table
Hist_table=HTable.read(592, with_units=False)
Hist_table


Month,Entries
IntegerAndSexagesimal,Historical
31 ;,"01s 00 ; 40,14"
59 ;,"02s 01 ; 20,28"
90 ;,"03s 02 ; 00,41"
120 ;,"04s 02 ; 40,55"
151 ;,"05s 03 ; 21,09"
181 ;,"06s 04 ; 01,23"
212 ;,"07s 04 ; 41,36"
243 ;,"08s 05 ; 21,50"
273 ;,"09s 06 ; 02,04"


In [3]:
# Formating and initialising the Test tables

Test_recomp=Hist_table.copy()
Delta_recomp=Hist_table.copy()
for i in range(12):
    Test_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)
    Delta_recomp[i][1]=Historical(Sexagesimal('0;0,0'),2)

In [4]:
#mean motion parameter from the Parisian Alfonsine Tables
daily_mean_elong=Historical(Sexagesimal("12;11,26,41,37,51,50,39"),8)
daily_mean_moon_lat=Historical(Sexagesimal("13;13,45,39,22,25,53,45"),7)
REVO=Historical(Sexagesimal("6,0;0,0"),8)

#length of the synodic month
synodic_month=REVO/daily_mean_elong
synodic_month=Temporal(synodic_month,7)
synodic_month=synodic_month.decimal
synodic_month=float(synodic_month)


for i in range(12):
    with set_precision(tmode=TruncatureMode.TRUNC, pmode=4):

        # Computing the mean sun and mean moon for a given number of synodic years
        months=i+1
        mean_moon_lat=(months*synodic_month*daily_mean_moon_lat)%REVO
                       
        #filling the Test and Delta tables with the final precision
        with set_precision(tmode=TruncatureMode.ROUND, pmode=2):
            Test_recomp[i][1]=1*mean_moon_lat
            Delta_recomp[i][1]=60*60*(Hist_table[i][1]-Test_recomp[i][1])
Test_recomp

Month,Entries
IntegerAndSexagesimal,Historical
31 ;,"01s 00 ; 40,14"
59 ;,"02s 01 ; 20,28"
90 ;,"03s 02 ; 00,41"
120 ;,"04s 02 ; 40,55"
151 ;,"05s 03 ; 21,09"
181 ;,"06s 04 ; 01,23"
212 ;,"07s 04 ; 41,36"
243 ;,"08s 05 ; 21,50"
273 ;,"09s 06 ; 02,04"


In [5]:
# residuals visualisation and statistical summaries 1
B=Delta_recomp.to_pandas()
(B.mean().mean(), B.std().std())

(0.0, nan)

In [6]:
B.astype(int).style.background_gradient(axis=None)

,Entries
Month,
31 ;,0
59 ;,0
90 ;,0
120 ;,0
151 ;,0
181 ;,0
212 ;,0
243 ;,0
273 ;,0


In [13]:
#csv export of the recomputed table
B=Test_recomp.to_pandas()
compression_opts = dict(method='zip',archive_name='Mean syzygies arg lat months.csv')
B.to_csv('Mean syzygies arg lat months.zip', index=False, compression=compression_opts)